In [1]:
import numpy as np
from environment import robot_env
import random
from IPython.display import clear_output
import time

In [2]:
#inisialisasi hyperparameter
num_episodes = 1000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
epsilon_decay_rate = 0.1

#Membuat Environment
env = robot_env.initialize_env()

#menginisialisasi posisi robot pertama kali (row_start, row_end, env)
env = robot_env.initialize_position(0, 0, env)

#Membuat array yang berisikan action apa saja yang bisa diambil
actions = robot_env.action_space()
# kiri = 0, kanan = 1, atas = 2, bawah = 3

#Initiaslisasi Q-Table
q_table = np.zeros((env.size, actions.size))

#inisialisasi reward
#Reward :
#Empty = -1
#Fire = -10
#2 Harta = +25
#1 Harta = +10
reward_all_episode = []


In [3]:
def choose_action(epsilon, state):
    random_num = random.uniform(0, 1)
    if (random_num < epsilon):
        action = np.random.choice(actions, 1)[0]
        
    else:
        index_state = state[0] + (state[0]*3) + state[0]
        action = np.argmax(q_table[index_state, :])
       
    return action

def update_q_table(state_, new_state):
    index_state = state_[0] + (state_[0]*3) + state_[0]
    index_new_state = new_state[0] + (new_state[0]*3) + new_state[0]
    
    
    q_table[index_state, action] = q_table[index_state, action] + (learning_rate * (reward + (discount_rate * np.max(q_table[index_new_state, :]) - q_table[index_state, action])))

In [4]:
for episode in range(num_episodes):
    env = robot_env.initialize_env()
    env = robot_env.initialize_position(0, 0, env)
    state = (0,0)
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        #choose action  (epislon greedy)
        action = choose_action(epsilon, state)
        #take action and get reward and next state
        new_state, reward, done, env = robot_env.take_action(action, env, False)
        #Uncommend this code if you render the environment
#         clear_output(wait=True)
#         time.sleep(0.2)
        #update q table
        update_q_table(state, new_state)
        #change current state
        rewards_current_episode += reward 
        
        #check if already on termination state
        if done:
            break
        state = new_state
    
    # Exploration rate decay
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
    
    #Add current episode reward to all episode reward
    reward_all_episode.append(rewards_current_episode)

In [5]:
print(q_table)

[[ 1.13691543e+02  7.13354800e+01 -1.89270848e-01  1.54737436e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.31617637e+02  1.57337433e+03  4.50811316e+02  9.58205598e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.00000000e-01 -1.00000000e-01  2.09285764e+02 -1.90000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
